In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("nlp_class.csv")

In [2]:
df

,Unnamed: 0,document,category
0,0,Irish markets reach all-time high,business
1,2,"Irish shares have risen to a record high, with...",business
2,4,The ISEQ index of leading shares closed up 23 ...,business
3,6,Several of Ireland's biggest companies saw the...,business
4,8,"Telecoms firm Eircom, which recently revealed ...",business
...,...,...,...
15129,23511,France has become the last big European countr...,tech
15130,23513,"Initially, more than a third of the population...",tech
15131,23515,"The French DTT platform is known as TNT, which...",tech
15132,23517,"""But everyone's impressed with DTT in the UK, ...",tech


In [3]:
df_business = df.loc[df['category']=='business']
df_business_text = list(df_business['document'])
df_business_text=' '.join(df_business_text)

df_tech = df.loc[df['category']=='tech']
df_tech_text = list(df_tech['document'])
df_tech_text=' '.join(df_tech_text)

df_sport = df.loc[df['category']=='sport']
df_sport_text = list(df_sport['document'])
df_sport_text=' '.join(df_sport_text)

df_entertainment = df.loc[df['category']=='entertainment']
df_entertainment_text = list(df_entertainment['document'])
df_entertainment_text=' '.join(df_entertainment_text)



In [4]:
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
wordcloud = WordCloud(width = 1000, height = 500, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(df_business_text)
plt.figure(figsize=(20,15))
plt.imshow(wordcloud)
plt.axis('off')

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(df_tech_text)
plt.figure(figsize=(20,15))
plt.imshow(wordcloud)
plt.axis('off')

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(df_sport_text)
plt.figure(figsize=(20,15))
plt.imshow(wordcloud)
plt.axis('off')

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(df_entertainment_text)
plt.figure(figsize=(20,15))
plt.imshow(wordcloud)
plt.axis('off')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk import stem
from nltk.corpus import stopwords
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

In [ ]:
df['document'][2]

In [ ]:
import re
def clean_text(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = text.lower()
  text = [x for x in text.split(' ') if x not in stopwords]
  text = ' '.join([stemmer.stem(word) for word in text])
  return text

In [ ]:
df['document']=df['document'].apply(str)

In [ ]:

df['document'] = df['document'].apply(clean_text)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['document'],df['category'],test_size = 0.2,random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 1000
max_len = 30
dimension = 8

In [ ]:
tokenizer = Tokenizer(num_words=1000,oov_token='<OOV>')

In [ ]:
tokenizer.fit_on_texts(X_train)

In [ ]:
#turn them into sequences
sequence_words = tokenizer.texts_to_sequences(X_train)

In [ ]:
seq_padded = pad_sequences(sequence_words,maxlen = max_len,padding='post',truncating='post')

In [ ]:
sequence_words_test = tokenizer.texts_to_sequences(X_test)
seq_padded_test = pad_sequences(sequence_words_test,maxlen = max_len,padding='post',truncating='post')

In [ ]:
label_tok = Tokenizer()
label_tok.fit_on_texts(y_train)
sequence_label = np.array(label_tok.texts_to_sequences(y_train))
sequence_label_test = np.array(label_tok.texts_to_sequences(y_test))

In [ ]:
#build LSTM
from keras.utils import to_categorical
sequence_label  = to_categorical(sequence_label)
sequence_label_test  = to_categorical(sequence_label_test)

In [ ]:

model = tf.keras.Sequential([
              tf.keras.layers.Embedding(vocab_size,dimension),
              #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(dimension)),
              #tf.keras.layers.Dense(dimension,activation='relu'),
              tf.keras.layers.LSTM(dimension),
              tf.keras.layers.Dense(dimension,activation='relu'),
              
              tf.keras.layers.Dense(6,activation='softmax')
])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


In [ ]:
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
his = model.fit(seq_padded,sequence_label,epochs = 20,validation_data=(seq_padded_test,sequence_label_test),verbose=2)

In [ ]:
df.groupby('category').describe().T